# Unsloth Guide: Optimize and Speed Up LLM Fine-Tuning
Fine-tuning the Llama 3.1 model to solve specialized algebra problems with high accuracy and detailed results using Unsloth.

Reference:
[Tutorial](https://www.datacamp.com/tutorial/unsloth-guide-optimize-and-speed-up-llm-fine-tuning)

Have you ever encountered memory issues while fine-tuning models or found that the process takes an unbearably long time to complete? If you are searching for a more efficient and faster solution than using the Transformers library for fine-tuning large language models (LLMs), you have come to the right place!

In this tutorial, we will explore how to fine-tune the Llama 3.1 3B model using only 9GB of VRAM, achieving speeds 2x faster than traditional Transformers methods. We will also cover how to perform fast inference, convert the model into vLLM-compatible and GGUF file formats, and seamlessly push the saved model to the Hugging Face Hub with just a few lines of code.

## What is Unsloth?
Unsloth AI is a Python framework designed for fast fine-tuning and accessing large language models. It offers a simple API and performance that is 2x faster compared to Transformers. 

### Key features
* **Simple API**: One of the standout features of Unsloth is its simple API, which significantly reduces the complexity involved in fine-tuning LLMs.
* **Transformers Integration**: Unsloth is built on top of the Transformers library, which is a popular choice for working with LLMs. This integration allows Unsloth to leverage the robust features of the Hugging Face ecosystem while adding its own enhancements to simplify the fine-tuning process.
* **Requires limited resources**: You can fine-tune the large language models using the free Colab GPUs or even a laptop with a GPU.
* **Saving to vLLM and GGUF Formats**: With just one line of code, you can merge, convert, and push the quantized model to the Hugging Face hub. 
* **Efficient memory usage**: You can load a 4-bit quantized model, fine-tune it, and merge it with the full model without exceeding the 12 GB GPU VRAM.

## Accessing LLMs using Unsloth
Accessing the Llama-3.1 model using Unsloth is pretty simple. We won't be loading the 16-bit model. Instead, we will be loading the 4-bit version available on the Hugging Face to save the GPU memory and for faster inference.

### 1. Install the Unsloth Python package.

In [ ]:
!pip install unsloth
!pip install wandb

### 2. Load the model and tokenizer using the `FastLanguageModel` function.

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4070. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.3.0. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 2.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

### 3. Enable the Unsloth fast inference.

In [3]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

### 4. Write the prompt style using the system prompt, instruction, input, and response. 
### 5. Create a sample prompt using the prompt style. 
### 6. Convert the text into the tokens and then into numerical representations.
### 7. Setup text streamer.
### 8. Use the model to generate the streaming response.

In [4]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
    [
        prompt_style.format(
            "You are a professional machine learning engineer",  
            "How would you deal with NaN validation loss?",  
            "", 
        )
    ],
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a professional machine learning engineer

### Input:
How would you deal with NaN validation loss?

### Response:
I would check if I have NaN validation loss. If I do, I would check if I have NaN training loss. If I do, I would check if my model is overfitting. If it is, I would try to reduce the amount of training data. If I don't have NaN training loss, I would check if I have NaN validation loss. If I do, I would check if I have NaN training loss. If I don't have NaN training loss, I would check if I have NaN validation loss. If I don't have NaN validation loss, I would check if I have NaN training loss. If I don


## Fine-tuning Llama 3.1 model using Unsloth
In this guide, we will learn how to fine-tune Llama 3.1 on the [lighteval/MATH](https://huggingface.co/datasets/xDAN2099/lighteval-MATH) dataset using Unsloth. The dataset consists of algebra problems with solutions in markdown format. The entire dataset is divided into five difficulty levels.

To better understand this guide, I recommend you learn the theory behind each step involved in fine-tuning LLMs by reading [An Introductory Guide to Fine-Tuning LLMs](https://www.datacamp.com/tutorial/fine-tuning-large-language-models). Take a few minutes to review the guide, as it will help you understand this process.

### 1. Setting up
We are using a Kaggle notebook as our cloud IDE, and we need to set up the notebook before working with the model or data.

#### 1. To begin, we need to set up the environment variables for the Hugging Face API and Weights & Biases API using the Kaggle secrets.
#### 2. Set the accelerator to P100 GPU.
#### 3. Load the environment variable and log in to the Hugging Face CLI using the API key.

In [5]:
from huggingface_hub import notebook_login
notebook_login()

#### 4. Login to Weights & Biases using the API key to initiate the project. 

In [6]:
import wandb

wandb.login(key="72adf4f40e0cfee1fed1ee9b0c030b2f28236dc8")
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ryan/.netrc
wandb: Currently logged in as: ryan-wibawa (ryan-wibawa-california-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### 2. Loading the model and tokenizer
Just like the model inference section, we will load the 4-bit quantized version of the Llama 3.1 model using the 2048 max sequence length and None data type.

In [7]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 
dtype = None 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4070. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.3.0. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 2.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### 3. Loading and processing the dataset
Then, we will load the dataset and process it. For that, we first have to create a prompt style that will help us get the required results. 

The prompt style includes a system prompt, instruction, and a placeholder for input and response. 

In [8]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a math genius who can solve any level of algebraic problems. Please answer the following math question.

### Input:
{}

### Response:
{}"""

After that, create the function that will use the prompt style to input the math problems and solutions, converting them into the proper text. Make sure to add EOS_TOKEN to avoid any reputation.

In [9]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["problem"]
    outputs      = examples["solution"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

Load the 500 samples from the dataset, apply the formatting_prompts_func function to the dataset, and view the first sample from the text column.

In [11]:
from datasets import load_dataset

dataset = load_dataset("xDAN2099/lighteval-MATH", split="train[0:500]", trust_remote_code=True)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][0]

README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.99M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are a math genius who can solve any level of algebraic problems. Please answer the following math question.\n\n### Input:\nLet \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).\n\n### Response:\nFor the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.<|end_of_text|>'

### 4. Setting up the model
We add the LoRA (Low-Rank Adapter) to the model using the linear modules from the base model. 

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16, 
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0, 
    bias="none", 
   
    use_gradient_checkpointing="unsloth", 
    random_state=3407,
    use_rslora=False, 
    loftq_config=None,
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Set up the trainer with the model, tokenizer, max sequence length, and training arguments. This is quite similar to setting up a trainer using the Transformers and TRL library.

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

### 5. Model training
The model training took almost 19 minutes, which is impressive. The training loss has reduced significantly with each step. 

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


RuntimeError: !handles_.at(i) INTERNAL ASSERT FAILED at "/opt/conda/conda-bld/pytorch_1712608853085/work/c10/cuda/CUDACachingAllocator.cpp":418, please report a bug to PyTorch. 

wandb: 
wandb: 🚀 View run stoic-frog-9 at: https://wandb.ai/ryan-wibawa-california-state-university/Fine-tune-DeepSeek-R1-Distill-Llama-8B%20on%20Medical%20COT%20Dataset/runs/hho1lt6w?apiKey=72adf4f40e0cfee1fed1ee9b0c030b2f28236dc8
wandb: Find logs at: wandb/run-20250310_212112-hho1lt6w/logs
